## LFR benchmark for Altmap vs Map Eq
### Compare altmap to map eq using networkx

In [70]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict

# show plots in separate window
%pylab
# load helpers and wrappers
%run helpers.py 

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [114]:
from networkx.generators.community import LFR_benchmark_graph
from sklearn.metrics import normalized_mutual_info_score as nmi_score
from sklearn.metrics import adjusted_mutual_info_score as ami_score

# generate LFR benchmark graph + extract ground truth communities
def generate_LFR_benchmark(N = 500, mu = 0.1):
    
    # LFR params N=10000
    # params = {'max_degree':50, 'max_community':100, 'min_community':50, 'average_degree':20, 'tau1':3.0, 'tau2':1.5}
    
    # LFR params generic
    max_community = int(0.2*N)
    min_community = int(max_community * 0.25) #0.5
    max_degree = int(max_community * 0.3) # 0.6
    min_degree = int(min_community * 0.4)
    gamma = 3.5 # Power law exponent for the degree distribution 
    beta = 1.1 #3.0 # Power law exponent for the community size distribution
    params = {'max_degree':max_degree, 'max_community':max_community, 'min_community':min_community,
              'min_degree':min_degree, 'tau1':gamma, 'tau2':beta}
    

    max_degree = params['max_degree']
    max_community = params['max_community']
    min_community = params['min_community']
    #average_degree = params['average_degree']
    min_degree = params['min_degree']
    tau1 = params['tau1'] # Power law exponent for the degree distribution 
    tau2 = params['tau2'] # Power law exponent for the community size distribution

    # generate LFR benchmark graph
    
    G = LFR_benchmark_graph(N, tau1, tau2, mu, min_degree=min_degree, max_degree=max_degree, 
                            max_community=max_community, min_community=min_community)
    G = nx.convert_node_labels_to_integers(G, first_label=1)
    
    # extract ground truth communities from networkx graph object
    communities_true = {}
    num_communities = 0
    for n in range(1,N+1):
        if n in communities_true:
            continue
            
        num_communities = num_communities + 1
        community = G.nodes[n]['community']
        node_ids = np.asarray(list(community))
        node_ids = node_ids + 1 # have node labels >= 1
        communities_true.update(dict.fromkeys(node_ids , num_communities))
        
    communities_true = OrderedDict(sorted(communities_true.items()))
    num_communities_true = max(communities_true.values()) - min(communities_true.values()) + 1
    
    return G, communities_true, num_communities_true

# compute normalized mutual information between two partitions
def compute_score(communities_true, communities_found):
    labels_true = list(communities_true.values())
    labels_found = list(communities_found.values())

    # return nmi_score(labels_true,labels_found, average_method='arithmetic')
    return ami_score(labels_true,labels_found, average_method='arithmetic')

class BenchmarkResults:
    def __init__(self, var_list, num_realizations):
        self.var_list = var_list
        self.num_datapoints = len(var_list)
        self.num_realizations = num_realizations
        self.actual_realizations = np.zeros((self.num_datapoints,), dtype=int)
        self.scores = np.zeros((num_realizations, self.num_datapoints))
        self.errors = np.zeros((num_realizations, self.num_datapoints))
    
    def evaluate_results(self):
        self.mean_scores = np.empty((self.num_datapoints,))
        self.mean_scores[:] = np.nan
        self.std_scores = np.empty((self.num_datapoints,))
        self.std_scores[:] = np.nan
        self.mean_errors = np.empty((self.num_datapoints,))
        self.mean_errors[:] = np.nan
        self.std_errors = np.empty((self.num_datapoints,))
        self.std_errors[:] = np.nan
        
        for i in range(self.num_datapoints):
            nr = self.actual_realizations[i]
            if nr == 0:
                continue
            
            self.mean_scores[i] = np.mean(self.scores[:nr, i])
            self.std_scores[i] = np.std(self.scores[:nr, i], ddof=1)
            self.mean_errors[i] = np.mean(self.errors[:nr, i])
            self.std_errors[i] = np.std(self.errors[:nr, i], ddof=1)
    
    def write_csv(self, path):
        df = pd.DataFrame()
        
        df['var_list'] = self.var_list
        df['actual_realizations'] = self.actual_realizations
        df['mean_scores'] = self.mean_scores
        df['std_scores'] = self.std_scores
        df['mean_errors'] = self.mean_errors
        df['std_errors'] = self.std_errors
        
        df.to_csv(path, index_label='id')
            
            
        
# LFR Benchmark
# num_realizations .. number of network realizations for each parameter pair (mu, N)
def run_benchmark(N, mu_list, num_realizations=10):
    num_benchmarks = len(mu_list) * num_realizations
    benchmark_id = 0
           
    infomap_results = BenchmarkResults(mu_list, num_realizations)
    altmap_results = BenchmarkResults(mu_list, num_realizations)
    altmap_sci_results = BenchmarkResults(mu_list, num_realizations)
    sci_results = BenchmarkResults(mu_list, num_realizations)
    for mu_idx, mu in enumerate(mu_list):
        
        realization_idx = -1
        for _ in range(num_realizations):
            benchmark_id = benchmark_id + 1
            print(f'Starting benchmark {benchmark_id}/{num_benchmarks} for (N,mu) = ({N},{mu})\n')
            try:
                G, communities_true, num_communities_true = generate_LFR_benchmark(N, mu)
            except nx.ExceededMaxIterations as err:
                print(f'Failed to generate network for (N,mu) = ({N},{mu}): ', err)
                continue
            
            realization_idx += 1
            
            # test infomap
            communities_found, num_communities_found, _, _ = infomap(G, altmap=False)
            print (f'Infomap found {num_communities_found} communities vs. {num_communities_true} ground truth '
                   f'communities.\n')
            
            score = compute_score(communities_true, communities_found)
            infomap_results.scores[realization_idx, mu_idx] = score
            error = num_communities_found/num_communities_true - 1.0
            infomap_results.errors[realization_idx, mu_idx] = error
            
            # test altmap
            communities_found, num_communities_found, _, _ = infomap(G, altmap=True, update_inputfile=False)
            print (f'Altmap found {num_communities_found} communities vs. {num_communities_true} ground truth '
                   f'communities.\n')
            
            score = compute_score(communities_true, communities_found)
            altmap_results.scores[realization_idx, mu_idx] = score
            error = num_communities_found/num_communities_true - 1.0
            altmap_results.errors[realization_idx, mu_idx] = error
        
            # test altmap with SCI
            communities_found, num_communities_found,\
            communities_init, num_communities_init = infomap(G, altmap=True, init='sc', update_inputfile=False)
            print (f'Altmap with SCI ({num_communities_init}) found {num_communities_found} communities vs. '
                   f'{num_communities_true} '
                   f'ground truth communities.\n')
            
            score = compute_score(communities_true, communities_found)
            altmap_sci_results.scores[realization_idx, mu_idx] = score
            error = num_communities_found/num_communities_true - 1.0
            altmap_sci_results.errors[realization_idx, mu_idx] = error
            
            score = compute_score(communities_true, communities_init)
            sci_results.scores[realization_idx, mu_idx] = score
            error = num_communities_init/num_communities_true - 1.0
            sci_results.errors[realization_idx, mu_idx] = error
        
        # store actual number of realizations    
        infomap_results.actual_realizations[mu_idx] = realization_idx + 1
        altmap_results.actual_realizations[mu_idx] = realization_idx + 1
        altmap_sci_results.actual_realizations[mu_idx] = realization_idx + 1
        sci_results.actual_realizations[mu_idx] = realization_idx + 1

    print(f'Finished benchmark successfully!\n')
    return infomap_results, altmap_results, altmap_sci_results, sci_results

def plot_benchmark_results(results: BenchmarkResults, type='scores', color='blue', marker=None, label=None, 
                           lower_bound=None, upper_bound=None):
    
    xdata = results.var_list
    data = results.mean_scores
    data_std = results.std_scores
    
    if type == 'errors':
        data = results.mean_errors
        data_std = results.std_errors
        
    lw = 2; ms = 10
    plt.plot(xdata, data, '--', marker = marker, color=color, linewidth=lw, markersize=ms, label=label)
    upper = data + data_std
    lower = data - data_std
    if lower_bound != None:
        lower[lower < lower_bound] = lower_bound
    if upper_bound != None:
        upper[upper > upper_bound] = upper_bound
        
    plt.fill_between(xdata, upper, lower, color=color, alpha=0.25)
    

In [105]:
import warnings
warnings.filterwarnings('ignore')

# benchmark params
N = 500
mu_list = np.linspace(0.15, 0.75, 5)
num_realizations = 3
print (mu_list)

infomap_results, altmap_results, altmap_sci_results, sci_results = run_benchmark(N, mu_list, num_realizations)

[0.15 0.3  0.45 0.6  0.75]
Starting benchmark 1/15 for (N,mu) = (500,0.15)

Infomap found 9 communities vs. 9 ground truth communities.

Altmap found 9 communities vs. 9 ground truth communities.

Spectral clustering finished in 0.7457770000000892 seconds.
Altmap with SCI (22) found 9 communities vs. 9 ground truth communities.

Starting benchmark 2/15 for (N,mu) = (500,0.15)

Infomap found 9 communities vs. 9 ground truth communities.

Altmap found 9 communities vs. 9 ground truth communities.

Spectral clustering finished in 0.3566960000002837 seconds.
Altmap with SCI (22) found 9 communities vs. 9 ground truth communities.

Starting benchmark 3/15 for (N,mu) = (500,0.15)

Infomap found 8 communities vs. 8 ground truth communities.

Altmap found 8 communities vs. 8 ground truth communities.

Spectral clustering finished in 0.45636700000022756 seconds.
Altmap with SCI (22) found 8 communities vs. 8 ground truth communities.

Starting benchmark 4/15 for (N,mu) = (500,0.3)

Infomap foun

In [111]:
infomap_results.evaluate_results()
altmap_results.evaluate_results()
altmap_sci_results.evaluate_results()
sci_results.evaluate_results()

print (f'Minimum number of realizations is {np.min(infomap_results.actual_realizations)}.')
print (f'Actual realizations are {infomap_results.actual_realizations}.')

Minimum number of realizations is 3.
Actual realizations are [3 3 3 3 3].


In [115]:
infomap_results.write_csv('./lfr_results/infomap_500n.csv')
altmap_results.write_csv('./lfr_results/altmap_500n.csv')
altmap_sci_results.write_csv('./lfr_results/altmap_sci_500n.csv')
sci_results.write_csv('./lfr_results/sci_500n.csv')


In [116]:
plt.close('all')
plt.figure()
# plt.title(f'LFR benchmark, N = {N} nodes')
plt.plot([0.5, 0.5], [0,1], 'r')

plot_benchmark_results(infomap_results, color='royalblue', marker='x', label='Infomap', lower_bound=0.0, 
                       upper_bound=1.0)
# plot_benchmark_results(sci_results, color='crimson', marker='s', label='SCI')
plot_benchmark_results(altmap_results, color='darkorange', marker='^', label='Synthesizing Infomap')
plot_benchmark_results(altmap_sci_results, color='seagreen', marker='o', label='Synthesizing Infomap with SCI')

plt.grid()
plt.xlabel('Mixing parameter $\mu$')
plt.ylabel(r'$\mathrm{AMI}(\mathcal{Y},\mathcal{Y}_{true})$')
plt.legend()

In [121]:
min_err = np.min(np.min([infomap_results.mean_errors - infomap_results.std_errors,\
                         altmap_results.mean_errors - altmap_results.std_errors,\
                         altmap_sci_results.mean_errors - altmap_sci_results.std_errors]))

max_err = np.max(np.max([infomap_results.mean_errors + infomap_results.std_errors,\
                         altmap_results.mean_errors + altmap_results.std_errors,\
                         altmap_sci_results.mean_errors + altmap_sci_results.std_errors]))
                                                  
plt.close('all')
plt.figure()
# plt.title(f'LFR benchmark, N = {N} nodes')
plt.plot([0.5, 0.5], [min_err-0.2, max_err+0.2], 'r')

plot_benchmark_results(infomap_results, type='errors', color='royalblue', marker='x', label='Infomap', lower_bound=-1.0)
# plot_benchmark_results(sci_results, type='errors', color='crimson', marker='s', label='SCI')
plot_benchmark_results(altmap_results, type='errors', color='darkorange', marker='^', label='Synthesizing Infomap')
plot_benchmark_results(altmap_sci_results, type='errors', color='seagreen', marker='o', label='Synthesizing Infomap '
                                                                                              'with SCI')
plt.grid()
plt.xlabel('Mixing parameter $\mu$')
plt.ylabel(r'Mean relative error $\bar{e}_\theta$')
plt.legend(loc='upper left')

In [28]:
# LFR params
N = 5000
mu = 0.01
max_community = int(0.5*N)
min_community = int(max_community * 0.25)
max_degree = int(max_community * 0.3)
min_degree = int(min_community * 0.4)
gamma = 3.5 # Power law exponent for the degree distribution 
beta = 1.1 # Power law exponent for the community size distribution
max_iter = 500

# generate LFR benchmark graph
G = LFR_benchmark_graph(N, gamma, beta, mu, min_degree=min_degree, max_degree=max_degree, 
                        max_community=max_community, min_community=min_community, max_iters=max_iter, tol=1)


In [122]:
G, communities_true, num_communities_true = generate_LFR_benchmark(N=500, mu=0.35)

_, _, comm_sizes = nodes_per_community(communities_true)
print (comm_sizes)
# plt.close('all')
# drawNetwork(G, communities_true, labels=False)

communities_infomap, _, _, _ = infomap(G, altmap=False)
communities_altmap, _, _, _ = infomap(G, altmap=True, update_inputfile=False)
communities_altmap_sci, _, communities_sci, _ = infomap(G, altmap=True, init='sc', update_inputfile=False)
#             
# print(f'Altmap cost for true partition is {altmap_cost(G, communities_true)}')

[36 97 30 55 56 92 39 95]
Spectral clustering finished in 0.8448590000002696 seconds.


In [123]:
# plotting params
node_size = 150
edge_width = 0.2
font_size = 13
# pos = nx.spring_layout(G)
# pos = nx.spectral_layout(G)
pos = community_layout(communities_true)


plt.close('all')
fig, axs = plt.subplots(2,2)

ax = axs[0,0]
ax.set_frame_on(False)
ax.set_yticklabels([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_title('Infomap')
communities = communities_infomap
color_idc = [v for v in communities.values()]
# nx.draw_networkx_edges(G, pos, ax=ax, width=edge_width)
nodeCollection = nx.draw_networkx_nodes(G, pos=pos, node_color=color_idc, cmap=plt.get_cmap('Set3'), ax=ax,
                                        node_size=node_size)
# nx.draw_networkx_labels(G, pos, ax=ax, labels=communities, font_weight='bold', font_size=font_size)


ax = axs[0,1]
ax.set_frame_on(False)
ax.set_yticklabels([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_title('Synthesizing Infomap')
communities = communities_altmap
color_idc = [v for v in communities.values()]
# nx.draw_networkx_edges(G, pos, ax=ax, width=edge_width)
nodeCollection = nx.draw_networkx_nodes(G, pos=pos, node_color=color_idc, cmap=plt.get_cmap('Set3'), ax=ax,
                                        node_size=node_size)
# nx.draw_networkx_labels(G, pos, ax=ax, labels=communities, font_weight='bold', font_size=font_size)


ax = axs[1,0]
ax.set_frame_on(False)
ax.set_yticklabels([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_title('Synthesizing Infomap with SCI')
communities = communities_altmap_sci
color_idc = [v for v in communities.values()]
# nx.draw_networkx_edges(G, pos, ax=ax, width=edge_width)
nodeCollection = nx.draw_networkx_nodes(G, pos=pos, node_color=color_idc, cmap=plt.get_cmap('Set3'), ax=ax,
                                        node_size=node_size)
# nx.draw_networkx_labels(G, pos, ax=ax, labels=communities, font_weight='bold', font_size=font_size)


ax = axs[1,1]
ax.set_frame_on(False)
ax.set_yticklabels([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_xticks([])
ax.set_title('Ground truth')
communities = communities_true
color_idc = [v for v in communities.values()]
# nx.draw_networkx_edges(G, pos, ax=ax, width=edge_width)
nodeCollection = nx.draw_networkx_nodes(G, pos=pos, node_color=color_idc, cmap=plt.get_cmap('Set3'), ax=ax,
                                        node_size=node_size)
# nx.draw_networkx_labels(G, pos, ax=ax, labels=communities, font_weight='bold', font_size=font_size)

